# Import netcdf files

In [1]:
from netCDF4 import Dataset
#import pandas as pd
import numpy as np
#import numpy.ma as ma
import vtk
from vtk.util import numpy_support
from pyproj import Transformer, transform
#from vtk.numpy_interface import dataset_adapter as dsa

In [2]:
src_nc = Dataset("../02_data/swc.nc", mode = "r", format = "NETCDF4")

In [3]:
print(src_nc.data_model)

NETCDF4


In [4]:
print(src_nc.groups)

OrderedDict()


In [5]:
print(src_nc.dimensions)

OrderedDict([('x', <class 'netCDF4._netCDF4.Dimension'>: name = 'x', size = 175
), ('y', <class 'netCDF4._netCDF4.Dimension'>: name = 'y', size = 225
), ('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 1318
)])


In [6]:
print(src_nc.variables)

OrderedDict([('lon', <class 'netCDF4._netCDF4.Variable'>
float64 lon(y, x)
    standard_name: longitude
    long_name: longitude
    units: degrees_east
    _CoordinateAxisType: Lon
unlimited dimensions: 
current shape = (225, 175)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('lat', <class 'netCDF4._netCDF4.Variable'>
float64 lat(y, x)
    standard_name: latitude
    long_name: latitude
    units: degrees_north
    _CoordinateAxisType: Lat
unlimited dimensions: 
current shape = (225, 175)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('time', <class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    standard_name: time
    long_name: time
    units: hours since 2016-1-1 00:00:00
    calendar: standard
    axis: T
unlimited dimensions: time
current shape = (1318,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('SWC_L01', <class 'netCDF4._netCDF4.Variable'>
float64 SWC_L01(time, y, x)
    long_name: soil water content of soil layer

In [7]:
print(src_nc.ncattrs)

<built-in method ncattrs of netCDF4._netCDF4.Dataset object at 0x7f91235c1228>


In [8]:
# access data of src_nc
# src.variables["var_name"][time][y][x]
print(len(src_nc.variables["SWC_L01"][0][:][1]))

175


/home/boog/software/anaconda3/envs/py3env/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  This is separate from the ipykernel package so we can avoid doing imports until


## Get variable data

### Get coordinates 

* this netcdf file uses coordinates latitude and longitude
* both coordiantes depend on the dimensions x and y
* x (175 elements) and y (225 elements) create a grid of 175*225 elements
* each element has a value for one of the variables lat, lon, scw_l01, scw_l02
* get lat and lon out

In [9]:
lat_dat = src_nc.variables["lat"][:]

# this procuces a masked array, --> use .filled()
lat_dat = lat_dat.filled()
lat_dat.shape[0]

225

In [10]:
lon_dat = src_nc.variables["lon"][:]

# this procuces a masked array, --> use .filled()
lon_dat = lon_dat.filled()
lon_dat.shape

(225, 175)

### Get time

In [11]:
time = src_nc.variables["time"]
time

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    standard_name: time
    long_name: time
    units: hours since 2016-1-1 00:00:00
    calendar: standard
    axis: T
unlimited dimensions: time
current shape = (1318,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [12]:
time.units

'hours since 2016-1-1 00:00:00'

In [13]:
# get time data
time = time[:]

### Get swc data

In [14]:
# get swc_l01
swcl01 = src_nc.variables["SWC_L01"]
swcl02 = src_nc.variables["SWC_L02"]

In [15]:
# get data from variable
# swcl01[:] will produce a masked arraya
# use .filled() to add the fill values in the array
swcl01_dat = swcl01[:].filled()
swcl02_dat = swcl02[:].filled()
type(swcl01_dat)

/home/boog/software/anaconda3/envs/py3env/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  after removing the cwd from sys.path.
/home/boog/software/anaconda3/envs/py3env/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  """


numpy.ndarray

## Transfrom to vtkStructuredGrid

### Def vtkStructuredGrid

In [16]:
# def point array
points = np.array([lon_dat.flatten(), 
                   lat_dat.flatten(),
                   np.repeat(0, len(lat_dat.flatten()))
                ]).transpose()
points

array([[ 4.80816264, 54.97939505,  0.        ],
       [ 4.87015916, 54.98306367,  0.        ],
       [ 4.9321697 , 54.98670084,  0.        ],
       ...,
       [15.00375566, 47.09808467,  0.        ],
       [15.05637693, 47.09669134,  0.        ],
       [15.10899422, 47.09527388,  0.        ]])

In [22]:
# def and fill vtkPoints
src_points = vtk.vtkPoints()
src_points.SetNumberOfPoints(len(points))

# define single point object
p = np.empty(3, dtype=np.float64)
p[2] = 0

# def coordinate transformer
transf = Transformer.from_crs(
                        "EPSG:4326",
                        "EPSG:5684",
                        always_xy=True)

for i in range(src_points.GetNumberOfPoints()):
    dest = transf.transform(points[i][0], points[i][1])
    p[0] = dest[0]
    p[1] = dest[1]
    src_points.SetPoint(i,p)

In [23]:
# create vtkStructuredGrid
src_grid = vtk.vtkStructuredGrid()
# be aware to set directions as x-y --> lon-lat not lat-lon
src_grid.SetDimensions(lat_dat.shape[1], lat_dat.shape[0],1)
src_grid.SetPoints(src_points)

# set points
# you have to use a loop to set individual points in vtk..really??
# src.GetPoints().SetPoint(i,p) where i is index, p is point tuple
# maybe do this when transorming the coordiante system, as in int_pol8.py
# I you have to use a for loop anyway
#src_grid.SetPoints(points);

In [24]:
#src_points = vtk.vtkPoints()
#src_points.SetNumberOfPoints(len(points))
#src_points.GetNumberOfPoints()
#src_points.SetPoint(1, [1,2,3])
#src_points.GetPoint(1)

### Add data

In [25]:
# use .flatten() to convert 2d array to 1d
new_point_arr_vtk1 = numpy_support.numpy_to_vtk(swcl01_dat[0].flatten())
new_point_arr_vtk1.SetName("SWC_L01")
new_point_arr_vtk2 = numpy_support.numpy_to_vtk(swcl02_dat[0].flatten())
new_point_arr_vtk2.SetName("SWC_L02")
src_grid.GetPointData().AddArray(new_point_arr_vtk1)
src_grid.GetPointData().AddArray(new_point_arr_vtk2)

1

#### Write test file

In [26]:
write_ouput = vtk.vtkXMLStructuredGridWriter()
write_ouput.SetFileName("dummy.vts")
write_ouput.SetInputData(src_grid) #int_out)
write_ouput.Write()

1